# Bayesian color constancy

In this notebook we follow the algorithm described in "Bayesian Color Constancy for Outdoor Object Recognition" by Tsin, Collins, Ramesh and Kanade.

We are estimating the following values:
* reflectance (sigma) per material
* light spectrum (beta) per light source
* material (o) per pixel
* light source (w) per pixel
* global illumination (g) per pixel

### Algorithm:

* find (sigma | o) and (beta | w) by LR decomposing labelled image patches
* find mean color for each combination of (o, w)
* for each pixel p:
    * get distance to each (o,w) pair
    * sort by distance
    * init hypotheses o, w, sigma, g
* for each light source w:
    * get pixel with best hypothesis w
    * estimate parameters for g(w)
    * init spectrum
* divide picture into windows
* until no hypothesis remains:
    * for each window:
        * get pixels with best hypothesis w
        * estimate parameters for g(w)
        * for each pixel p:
            * for each hypothesis H:
                * update sigma
                * update g
    * for each pixel p:
        * get likelihood
    * update light spectrum for each w
    * delete least likely hypothesis for each pixel


In [ ]:
# INPUTS:

import glob
import itertools
import math
import os

import cv2
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg

INDEX_BEST_HYPOTHESIS = 0

NUM_COLORS = 3

### The following inputs are required and are currently set manually in code:
* potential materials o
* prior for each material
* potential light sources w
* prior for each light sourc
* representative image regions for o
* representative images w
* table containing the mean color for each light source / material combination

In [ ]:
MATERIALS = ["LEAF",
             "CEMENT",
             "PIPE",
             "LID"]
NUM_MATERIALS = len(MATERIALS)
p_o = np.ndarray((NUM_MATERIALS,))
p_o[:] = 1. / NUM_MATERIALS # currently

LIGHT_SOURCES = ["DIRECT_LIGHT",
                 "BRIGHT_LIGHT"]
NUM_LIGHT_SOURCES = len(LIGHT_SOURCES)
p_w = np.ndarray((NUM_LIGHT_SOURCES,))
p_w[:] = 1. / NUM_LIGHT_SOURCES


# MAKE SURE ALL IMAGE REGIONS ARE THE SAME SIZE AND RELATIVELY SMALL (20 x 20)
image_regions = {
    "LEAF": ((220, 420), (250, 450)),
    "CEMENT": ((80, 120), (110, 150)),
    "PIPE": ((325, 520), (355, 550)),
    "LID": ((0, 0), (30, 30)),
}

# MAKE SURE ALL EXAMPLE LIGHT SOURCE IMAGES ARE THE SAME SIZE
image_light_sources = {
    "DIRECT_LIGHT": [3, 5],
    "BRIGHT_LIGHT": [2, 6]
}

MATERIALS_LIGHT_SOURCE_COMBINATIONS = list(itertools.product(range(NUM_MATERIALS), range(NUM_LIGHT_SOURCES)))
NUM_HYPOTHESISES = len(MATERIALS_LIGHT_SOURCE_COMBINATIONS)
MODEL_DIMENSIONS = 2  # can be increased to make everything more accurate

sigma_p = np.diag([10, 10, 10]) # the correlation matrix between pixel values, can be obtained by running a seperate color calibration algorithm on the images.
invered_sigma_p = np.linalg.inv(sigma_p)

def get_mean_color_chart():
    mean_color_chart = np.ndarray((NUM_MATERIALS, NUM_LIGHT_SOURCES, NUM_COLORS))

    mean_color_chart[0, 0, :] = [92, 190, 120]
    mean_color_chart[1, 0, :] = [150, 220, 215]
    mean_color_chart[2, 0, :] = [80, 210, 240]
    mean_color_chart[3, 0, :] = [30, 200, 210]

    mean_color_chart[0, 1, :] = [250, 255, 255]
    mean_color_chart[1, 1, :] = [255, 255, 255]
    mean_color_chart[2, 1, :] = [210, 255, 255]
    mean_color_chart[3, 1, :] = [65, 255, 250]

    return mean_color_chart

First we build the distribution over the reflectance per material and the spectrum per light source.

In [ ]:

def get_l_r_decomposition(image_data):
    sample_image_patch = list(image_regions.values())[0]
    patch_width = (sample_image_patch[0][0] - sample_image_patch[1][0])
    patch_height = (sample_image_patch[0][1] - sample_image_patch[1][1])
    num_pixel_in_patch = patch_width * patch_height

    num_example_files = len(list(image_light_sources.values())[0])

    Rs = np.ndarray((NUM_MATERIALS, NUM_LIGHT_SOURCES, NUM_COLORS * MODEL_DIMENSIONS, num_pixel_in_patch))
    Ls = np.ndarray((NUM_MATERIALS, NUM_LIGHT_SOURCES, MODEL_DIMENSIONS, num_example_files))

    for material_index, corners in enumerate(image_regions.values()):
        p1, p2 = corners
        for light_source_index, image_list in enumerate(image_light_sources.values()):
            image_patch = image_data[image_list, p1[1]:p2[1], p1[0]:p2[0], :].reshape(num_example_files,
                                                                                      3 * num_pixel_in_patch)
            p, l, u = scipy.linalg.svd(image_patch, full_matrices=False)
            R = u
            L = p @ np.diag(l)
            Rs[material_index, light_source_index, :, :] = R[0:MODEL_DIMENSIONS, :].reshape(
                (3 * MODEL_DIMENSIONS, num_pixel_in_patch))
            Ls[material_index, light_source_index, :, :] = L[0:MODEL_DIMENSIONS, :]

    return Ls, Rs


def estimate_sigma_distribution(Rs):
    mu = np.ndarray((NUM_MATERIALS, MODEL_DIMENSIONS * NUM_COLORS))
    cov = np.zeros((NUM_MATERIALS, MODEL_DIMENSIONS * NUM_COLORS, MODEL_DIMENSIONS * NUM_COLORS))

    for o in range(NUM_MATERIALS):
        values = Rs[o, :, :].T @ p_w[:]
        mu[o, :, ] = values.mean(axis=0)
        cov[o, :, :] = np.cov(values.T)
    return mu, cov


def estimate_beta_hats(Ls):
    beta_hats = np.ndarray((NUM_LIGHT_SOURCES, MODEL_DIMENSIONS, NUM_FILES))

    beta_hats[:, :, 0] = (Ls[:, :, :, :].T @ p_o).T.mean(axis=2)
    for image_index in range(NUM_FILES):
        beta_hats[:, :, image_index] = (Ls[:, :, :, :].T @ p_o).T[:, :, image_index % Ls.shape[3]]

    return beta_hats / beta_hats.sum(axis=0)


def estimate_beta_distribution(beta_hats):
    mu = np.ndarray((NUM_LIGHT_SOURCES, MODEL_DIMENSIONS))
    cov = np.ndarray((NUM_LIGHT_SOURCES, MODEL_DIMENSIONS, MODEL_DIMENSIONS))

    for w in range(NUM_LIGHT_SOURCES):
        values = beta_hats[w, :, :]
        mu[w] = values.mean(axis=1)
        cov[w] = np.cov(values)
    return mu, cov

# we are not interested in the actual angle, just the relative ordering, so no root of the value
def color_angle_distance(color1, color2):
    total = 0
    for i in range(3):
        total += (color1[i] - color2[i]) ** 2
    return total


def color_metric(color):
    return math.sqrt(color_angle_distance([0, 0, 0], color))

Next we initialize all hypothesises (o/w pair) for all pixels with a value for the reflectance (sigma) and global illumination (g).

In [ ]:

def init_pixel_hypothesises(img):
    pixel_o = np.ndarray((NUM_PIXEL, NUM_HYPOTHESISES), dtype="uint8")
    pixel_w = np.ndarray((NUM_PIXEL, NUM_HYPOTHESISES), dtype="uint8")
    pixel_sigma = np.ndarray((NUM_PIXEL, NUM_HYPOTHESISES, MODEL_DIMENSIONS * 3))
    pixel_g = np.ndarray((NUM_PIXEL, NUM_HYPOTHESISES))

    for pi, pixel in enumerate(img[:, ]):
        distances = [color_angle_distance(pixel, mean_color_chart[o, w, :]) for o, w in
                     MATERIALS_LIGHT_SOURCE_COMBINATIONS]

        # sort all hypothesises by descending likelihood
        hypothesises = list(range(NUM_HYPOTHESISES))
        hypothesises.sort(key=lambda i: distances[i])

        for rank, h in enumerate(hypothesises):
            o = MATERIALS_LIGHT_SOURCE_COMBINATIONS[h][0]
            w = MATERIALS_LIGHT_SOURCE_COMBINATIONS[h][1]

            pixel_o[pi, rank] = o
            pixel_w[pi, rank] = w
            pixel_sigma[pi, rank, :] = Rs[o, w, :, :].mean(axis=1)
            pixel_g[pi, rank] = pixel @ mean_color_chart[o, w] / color_metric(mean_color_chart[o, w])

    return pixel_o, pixel_w, pixel_sigma, pixel_g

The function to estimate the global illumination (g) distribution per light source.

In [ ]:
def estimate_g_distriution(g, w):
    mu_g = np.ndarray((NUM_LIGHT_SOURCES,))
    std_g = np.ndarray((NUM_LIGHT_SOURCES,))

    for light_source in range(NUM_LIGHT_SOURCES):
        mu_g[light_source] = 0
        std_g[light_source] = 1

        pixels_predicting_light_source = [i for i, w_hat in enumerate(w) if w_hat == light_source]
        if len(pixels_predicting_light_source) != 0:
            gw = np.array([g[i] for i in pixels_predicting_light_source])
            mu_g[light_source] = gw.mean()
            std_g[light_source] = np.std(gw)

    return mu_g, std_g

The reflectance for each pixel and each hypothesis is updated in the following function:

In [ ]:
def update_sigma(sigma_p, g, beta, Sigma_o, mu_o, pixel):
    # the matrizes should have the following dimensions:
    # K1 : MODEL_DIMENSIONS x MODEL_DIMENSIONS
    # K2 : MODEL_DIMENSIONS x 3
    # K3 : 3 x 3

    K3 = np.linalg.inv(sigma_p / g ** 2 + beta.T @ Sigma_o @ beta)
    K2 = g * Sigma_o @ beta @ (np.eye(NUM_COLORS) - K3 @ beta.T @ Sigma_o @ beta)
    K1 = np.eye(NUM_COLORS * MODEL_DIMENSIONS) - Sigma_o @ beta @ K3 @ beta.T

    return K1 @ mu_o + K2 @ np.linalg.inv(sigma_p) @ pixel

After all pixel values were updated, we can update the distributions for the spectrum and the global illumination.

In [ ]:
def update_betas(pixel_w, pixel_sigma, pixel_g, beta_distribution_mean, beta_distribution_std, sigma_p):
    for light_source in range(NUM_LIGHT_SOURCES):
        pixels_predicting_light_source = [i for i, w_hat in enumerate(pixel_w[:, INDEX_BEST_HYPOTHESIS])
                                          if w_hat == light_source]
        h = np.zeros((MODEL_DIMENSIONS, MODEL_DIMENSIONS))
        b = np.zeros((MODEL_DIMENSIONS,))

        mu_beta = beta_distribution_mean[light_source, :]
        Sigma_beta = beta_distribution_std[light_source, :, :]
        Sigma_beta_inv = np.linalg.inv(Sigma_beta)

        # in case of large images, sampling from pixels_predicting_light_source will provide a speedup
        # like: pixels_predicting_light_source = pixels_predicting_light_source[::N]

        for pi in pixels_predicting_light_source:
            S = pixel_sigma[pi, 0].reshape((MODEL_DIMENSIONS, NUM_COLORS))
            pixel = img[pi, :]

            h += (pixel_g[pi, 0] ** 2) * S @ sigma_p @ S.T
            b += pixel_g[pi, 0] * S @ sigma_p @ pixel

        h += len(pixels_predicting_light_source) * Sigma_beta_inv
        b += len(pixels_predicting_light_source) * (Sigma_beta_inv @ mu_beta)

        beta_hats[light_source, :, image_index] = np.linalg.inv(h) @ b

    return beta_hats

def update_g(mu_g, cov_g, pixel, inverted_sigma_p, S, beta):
    g_nom = (mu_g[w] + cov_g[w] ** 2 * pixel @ inverted_sigma_p @ S.T @ beta)
    g_denom = (1 + (cov_g[w] ** 2) * beta @ S @ inverted_sigma_p @ S.T @ beta)

    return g_nom / g_denom

After all new values are calculated, we get the likelihood for each hypothesis for each pixel.
Since the values get small, log-likelihood is used instead.

In [ ]:

def log_N_multidim(value, mu, cov):
    dims = cov.shape[0]
    return -dims / 2 * math.log(2 * math.pi) - 0.5 * math.log(abs(np.linalg.det(cov))) \
           + (-0.5 * ((value - mu).T @ np.linalg.inv(cov) @ (value - mu)))


def log_N_singledim(value, mu, std):
    return -0.5 * math.log(2 * math.pi) - 0.5 * std \
           + (-0.5 * (1 / std) * (value - mu) ** 2)


def calc_log_likelihoods(img, num_hypothesises_remaining):
    # p(p|S,beta, g) * p(beta|w) * p(S|o) * p(g|w) * p(w) * p(o)

    likelihoods = np.ndarray((NUM_PIXEL, num_hypothesises_remaining))
    for pi, pixel in enumerate(img[:, ]):
        for hi in range(num_hypothesises_remaining):
            o = pixel_o[pi, hi]
            w = pixel_w[pi, hi]

            S = pixel_sigma[pi, hi].reshape((MODEL_DIMENSIONS, NUM_COLORS))

            p_p = log_N_multidim(pixel,
                                 pixel_g[pi, hi] * S.T @ beta_hats[w, :, image_index],
                                 sigma_p)
            p_S = log_N_multidim(pixel_sigma[pi, hi, :],
                                 sigma_distribution_mean[o, :],
                                 sigma_distribution_std[o])
            p_g = log_N_singledim(pixel_g[pi, hi],
                                  mu_g[w],
                                  std_g[w])
            p_beta = log_N_multidim(beta_hats[w, :, image_index],
                                    beta_distribution_mean[w],
                                    beta_distribution_std[w])
            likelihoods[pi, hi] = p_p + p_beta + p_S + p_g + p_w[w] + p_o[o]

    return likelihoods

We remove the least likely hypothesis.

In [ ]:
def update_hypothesises(likelihoods, pixel_o, pixel_w, pixel_sigma, pixel_g):
    num_hypothesises_remaining = pixel_o.shape[1]

    pixel_o_new = np.ndarray((NUM_PIXEL, num_hypothesises_remaining - 1), dtype="uint8")
    pixel_w_new = np.ndarray((NUM_PIXEL, num_hypothesises_remaining - 1), dtype="uint8")
    pixel_sigma_new = np.ndarray((NUM_PIXEL, num_hypothesises_remaining - 1, MODEL_DIMENSIONS * NUM_COLORS))
    pixel_g_new = np.ndarray((NUM_PIXEL, num_hypothesises_remaining - 1))

    for pi in range(NUM_PIXEL):
        # update arrays sorted by likelihood
        sorted_likelihoods = list(range(num_hypothesises_remaining))
        sorted_likelihoods = list(sorted(sorted_likelihoods, key=lambda i: likelihoods[pi, i]))[:-1]

        pixel_o_new[pi, :] = pixel_o[pi, sorted_likelihoods]
        pixel_w_new[pi, :] = pixel_w[pi, sorted_likelihoods]
        pixel_sigma_new[pi, :] = pixel_sigma[pi, sorted_likelihoods]
        pixel_g_new[pi, :] = pixel_g[pi, sorted_likelihoods]

    return pixel_o_new, pixel_w_new, pixel_sigma_new, pixel_g_new


A function for writing the segmented image for each material or light source.

In [ ]:
def write_segmented_image(region_x, region_y, mask, filename):
    masked = np.zeros((NUM_PIXEL, NUM_COLORS))
    masked[mask, :] = [255, 255, 255]
    masked = masked.reshape((IMAGE_HEIGHT, IMAGE_WIDTH, 3))
    masked = masked[region_y[0]:region_y[1], region_x[0]:region_x[1], :]
    cv2.imwrite(filename, masked)


def flatten_indizes(x_start, x_end, y_start, y_end, width):
    return [py * width + px
            for px in range(x_start, x_end)
            for py in range(y_start, y_end)]


Putting it all together. Load a folder full of images, initialize hypothesises and iterate until there is only one hypothesis left.

In [ ]:
def get_images_from_folder(pattern):
    files = glob.glob(pattern)
    num_files = len(files)

    img = cv2.imread(files[0])

    data = np.ndarray((num_files, img.shape[0], img.shape[1], 3))
    for i, f in enumerate(files):
        img = cv2.imread(f)
        data[i, :, :, :] = img[:, :, :]
    return data

print("START")

image_data = get_images_from_folder("../data/cropped_plants/*.jpg")

NUM_FILES = image_data.shape[0]
IMAGE_HEIGHT = image_data.shape[1]
IMAGE_WIDTH = image_data.shape[2]
NUM_PIXEL = image_data.shape[1] * image_data.shape[2]

print("SVD DECOMPOSITION OF IMAGE PATCHES")

Ls, Rs = get_l_r_decomposition(image_data)
sigma_distribution_mean, sigma_distribution_std = estimate_sigma_distribution(Rs)
beta_hats = estimate_beta_hats(Ls)
beta_distribution_mean, beta_distribution_std = estimate_beta_distribution(beta_hats)

print("FIND MEAN COLOR FOR EACH COMBINATION OF MATERIAL AND LIGHT SOURCE")

mean_color_chart = get_mean_color_chart()

# for now, only work on a single image
for image_index, f in enumerate(["../data/cropped_plants/1604404802.jpg"]):
    filename = os.path.splitext(os.path.basename(f))[0]
    img = cv2.imread(f)
    img = img.reshape((NUM_PIXEL, NUM_COLORS))
    img_width = IMAGE_WIDTH
    img_height = IMAGE_HEIGHT

    print(f"{f} INIT PIXEL HYPOTHESISES")

    pixel_o, pixel_w, pixel_sigma, pixel_g = init_pixel_hypothesises(img)

    print(f"{f} INIT LIGHT SOURCE HYPOTHESISES")

    mu_g, std_g = estimate_g_distriution(pixel_g[:, INDEX_BEST_HYPOTHESIS], pixel_w[:, INDEX_BEST_HYPOTHESIS])

    print(f"{f} ITERATE OVER IMAGE WINDOWS")

    # we take a part of the image (the macro region) and slide over it in overlapping windows
    # the windows should overlap a bit, the amount is set here
    MACRO_REGION_X = (0, 600)
    MACRO_REGION_Y = (000, 600)
    WINDOW_X_SIZE = 50
    WINDOW_Y_SIZE = 50
    OVERLAPPING = 25

    for iteration in range(NUM_HYPOTHESISES - 1):
        num_hypothesises_remaining = NUM_HYPOTHESISES - iteration

        print(f"{f} Run {iteration + 1} / {NUM_HYPOTHESISES - 1}")

        for window_x in range(MACRO_REGION_X[0], MACRO_REGION_X[1] + 1, WINDOW_X_SIZE - OVERLAPPING):
            for window_y in range(MACRO_REGION_Y[0], MACRO_REGION_Y[1] + 1, WINDOW_X_SIZE - OVERLAPPING):
                pixel_indizes_patch = flatten_indizes(window_x, min(window_x + WINDOW_X_SIZE, img_width),
                                                      window_y, min(window_y + WINDOW_Y_SIZE, img_height),
                                                      img_width)

                mu_g_patch, std_g_patch = estimate_g_distriution(pixel_g[pixel_indizes_patch, INDEX_BEST_HYPOTHESIS],
                                                                 pixel_w[pixel_indizes_patch, INDEX_BEST_HYPOTHESIS])

                for pi in pixel_indizes_patch:
                    pixel = img[pi]

                    for hi in range(num_hypothesises_remaining):
                        w = pixel_w[pi, hi]
                        o = pixel_o[pi, hi]
                        S = pixel_sigma[pi, hi].reshape((MODEL_DIMENSIONS, NUM_COLORS))

                        beta_old = beta_hats[w, :, image_index]

                        # pad the beta values into a larger matrix
                        beta_matrix = np.zeros((NUM_COLORS * MODEL_DIMENSIONS, NUM_COLORS))
                        for i in range(NUM_COLORS):
                            beta_matrix[i * MODEL_DIMENSIONS:(i + 1) * MODEL_DIMENSIONS, i] = beta_old

                        pixel_sigma[pi, hi] = update_sigma(sigma_p, pixel_g[pi, hi], beta_matrix,
                                                           sigma_distribution_std[o, :, :], sigma_distribution_mean[o],
                                                           pixel)

                        pixel_g[pi, hi] = update_g(mu_g_patch, std_g_patch, pixel, invered_sigma_p, S, beta_old)

        likelihoods = calc_log_likelihoods(img, num_hypothesises_remaining)

        beta_hats = update_betas(pixel_w, pixel_sigma, pixel_g, beta_distribution_mean, beta_distribution_std, sigma_p)

        pixel_o, pixel_w, pixel_sigma, pixel_g = update_hypothesises(likelihoods, pixel_o, pixel_w, pixel_sigma,
                                                                     pixel_g)

        for o in range(NUM_MATERIALS):
            write_segmented_image(MACRO_REGION_X, MACRO_REGION_Y, [pi for pi in range(NUM_PIXEL) if pixel_o[pi, INDEX_BEST_HYPOTHESIS] == o],
                                  f"{filename}_masked_by_object_{MATERIALS[o]}_{iteration}.png")

        for w in range(NUM_LIGHT_SOURCES):
            write_segmented_image(MACRO_REGION_X, MACRO_REGION_Y, [pi for pi in range(NUM_PIXEL) if pixel_w[pi, INDEX_BEST_HYPOTHESIS] == w],
                                  f"{filename}_masked_by_lightsource_{LIGHT_SOURCES[w]}_{iteration}.png")

    print(f"{f} PRINT SEGMENTED IMAGES")

    ll_image = likelihoods[:, INDEX_BEST_HYPOTHESIS].reshape((img_height, img_width))
    ll_image_region = ll_image[MACRO_REGION_Y[0]:MACRO_REGION_Y[1], MACRO_REGION_X[0]:MACRO_REGION_X[1]]
    plt.imshow(ll_image_region, cmap="hot")
    plt.show()

    leaf_likelihoods = likelihoods[[i for i in range(NUM_PIXEL) if pixel_o[i, INDEX_BEST_HYPOTHESIS] == 0], 0]
    plt.hist(leaf_likelihoods, bins=100, cumulative=True)
    plt.show()

print("END")
